In [25]:
import os
import pcapy as p
from scapy.all import *
import matplotlib.pyplot as plt

import pandas as pd

import struct
import numpy as np

In [26]:
data = "data/setup.pcapng"

In [27]:
packets = rdpcap(data)
Config = []
IMGS = []
IMGSsmol = []
idxImags = []
IMGS2 = []
IMGSsmol2 = []
idxImags2 = []
start = 0
ix = 1
for k in packets[52:359]:
    k = k[Raw].load
    EP = -1
    if k[8] == 0x53: # URB submit
        if k[9] == 0x02: # URB submit
            TYPE = "URB_CONTROL"
        elif k[9] == 0x03: # BULK
            TYPE = "BULK"
        EP = k[10] # EP
        if EP == 0:
            SENS ="OUT"
        else:
            SENS = "IN"
        if TYPE == "BULK":
            sizeRequested = k[33]*256+k[32] 
            Config.append({'ID': ix,"Sens":SENS,"TYPE":TYPE,"EP":k[10],"sizeRequested":sizeRequested})
        else:
            bmRequestType = k[40]
            bmRequest = k[41]
            bmValue  =  256*k[43]+k[42] 
            bmIndex  = k[44]+256*k[45]  
            bmLength =  k[46]+256*k[47]  
            if bmLength:
                PL = k[64:]
            else:
                PL = ""
            #print(ix,hex(bmRequestType),bmRequest,hex(bmValue),bmIndex,bmLength,TYPE,SENS,hex(EP))
            Config.append({'ID': ix,"Sens":SENS,"TYPE":TYPE,"EP":k[10],"bmRequestType":bmRequestType,
                           "bmRequest":bmRequest,"bmValue":bmValue,"bmIndex":bmIndex,"bmLength":bmLength,
                           "PL":PL
                      })
            
    if k[8] == 0x43: # URB receive
        if 0:
            if (len(k) == 65):
                print(ix,hex(k[-1]))
            if (len(k) == 66):
                print(ix,hex(k[-1]+256*k[-2]  ))
                
    if (k[8] == 0x43) & ((k[10] == 0x86)): # URB receive  
        if len(k) > 20000:
            IMGS.append(k[64:])
            #print(ix)
            idxImags.append(str(ix)+"full")
        else:
            IMGSsmol.append(k[64:])
            idxImags.append(str(ix)+"short")
    if (k[8] == 0x43) & ((k[10] == 0x82)): # URB receive  
        if len(k) > 20000:
            IMGS2.append(k[64:])
            #print(ix)
            idxImags2.append(str(ix)+"full")
        else:
            IMGSsmol2.append(k[64:])
            idxImags2.append(str(ix)+"short")   
    ix +=1

In [40]:
df = pd.DataFrame(Config)  
df["PLL"] = df.PL.apply(lambda x: len(x))
lV = df.bmValue.tolist()
lP = df.PL.tolist()
df

ID Sens         TYPE  EP  bmRequestType  bmRequest  bmValue  bmIndex  \
0      1  OUT  URB_CONTROL   0             64        160    58880        0   
1      3  OUT  URB_CONTROL   0             64        160       54        0   
2      5  OUT  URB_CONTROL   0             64        160     2452        0   
3      7  OUT  URB_CONTROL   0             64        160     2516        0   
4      9  OUT  URB_CONTROL   0             64        160     2580        0   
..   ...  ...          ...  ..            ...        ...      ...      ...   
148  297  OUT  URB_CONTROL   0             64        160     3266        0   
149  299  OUT  URB_CONTROL   0             64        160     3998        0   
150  301  OUT  URB_CONTROL   0             64        160     4062        0   
151  303  OUT  URB_CONTROL   0             64        160     4633        0   
152  305  OUT  URB_CONTROL   0             64        160    58880        0   

     bmLength                                                 PL  PLL  
0           1                                            b'\x01'    1  
1          10        b'\x00\x01\x02\x02\x03\x03\x04\x04\x05\x05'   10  
2          64  b'\xe4\xf5-\xf5,\xf5+\xf5*\xc2\x03\xc2\x00\xc2...   64  
3          64  b'\x80\x8e0\x8f1\xc3t\x90\x9f\xfft\x0f\x9e\xcf...   64  
4          64  b'\x82\xe505$\xf5\x83\xe0\xff\xe5/%%\xf5\x82\x...   64  
..        ...                                                ...  ...  
148        27  b'\x93p\r\xa3\xa3\x93\xf8t\x01\x93\xf5\x82\x88...   27  
149        64  b'\x02\t\x94\xe4\x93\xa3\xf8\xe4\x93\xa3@\x03\...   64  
150        64  b'\x01\x93`\xbc\xa3\xffT?0\xe5\tT\x1f\xfe\xe4\...   64  
151         1                                            b'\x00'    1  
152         1                                            b'\x00'    1  

[153 rows x 11 columns]

# Let's program the probe

In [34]:
#USB
import usb.core
import usb.util 
# Figures
import pandas as pd
import numpy as np
# Utilities
import json
import base64
import struct

In [39]:
dev = usb.core.find(idVendor=0x1921, idProduct=0x0001)
if not dev: print("No Device")

c = 0
IFS = []
for config in dev:
    print('config', c)
    print('Interfaces', config.bNumInterfaces)
    for iface in config:
        #print(iface)
        IFS.append(iface)
        #print(" TORLOLOLO \n\n")
    # The device was getting "Err 16 busy" on my ubuntu
    for i in range(config.bNumInterfaces):
        if dev.is_kernel_driver_active(i):
            dev.detach_kernel_driver(i)
        #print(i)
    c+=1 
dev.reset()

try:
    dev.set_configuration()
except:
    print("Already connected")
    

cfg = dev.get_active_configuration()
intf = cfg[(0,0)]



config 0
Interfaces 1


In [43]:
print(lP)

[b'\x01', b'\x00\x01\x02\x02\x03\x03\x04\x04\x05\x05', b'\xe4\xf5-\xf5,\xf5+\xf5*\xc2\x03\xc2\x00\xc2\x02\xc2\x01\x12\x08\x00~\x0f\x7f\x00\x8e=\x8f>uE\x0fuF\x12u;\x0fu<\x1cuC\x0fuD<uG\x0fuH\\\xeeT\xc0p\x02A\x94u.\x00u/', b'\x80\x8e0\x8f1\xc3t\x90\x9f\xfft\x0f\x9e\xcf$\x02\xcf4\x00\xfe\xe4\x8f)\x8e(\xf5\'\xf5&\xf5%\xf5$\xf5#\xf5"\xaf)\xae(\xad\'\xac&\xab%\xaa$\xa9#\xa8"\xc3\x12\x0c\xa6P3\xe51%%\xf5', b'\x82\xe505$\xf5\x83\xe0\xff\xe5/%%\xf5\x82\xe5.5$\xf5\x83\xef\xf0\xe5%$\x01\xf5%\xe45$\xf5$\xe45#\xf5#\xe45"\xf5"\x80\xb7\x85.=\x85/>t\x00$\x80\xfft\x0f4\xff\xfe\xc3\xe5', b'F\x9f\xf5F\xe5E\x9e\xf5E\xc3\xe5@\x9f\xf5@\xe5?\x9e\xf5?\xc3\xe5B\x9f\xf5B\xe5A\x9e\xf5A\xc3\xe5<\x9f\xf5<\xe5;\x9e\xf5;\xc3\xe5D\x9f\xf5D\xe5C\x9e\xf5C\xc3\xe5H\x9f\xf5H\xe5G\x9e\xf5G', b'\xd2\xe8C\xd8 \x90\xe6h\xe0D\t\xf0\x90\xe6\\\xe0D=\xf0\xd2\xaf\x90\xe6\x80\xe0 \xe1\x05\xd2\x08\x12\x13Z\x90\xe6\x80\xe0T\xf7\xf0S\x8e\xf8\xc2\x03\x12\x00\n0\x01\x05\x12\x03\xdf\xc2\x010\x03\xf2\x12\x12\xc0P\xed', b'\xc2\x03\x12\x14

In [42]:
for k in range(len(lV)):
    if  row["Sens"] == "OUT":
        dev.ctrl_transfer(bmRequestType= 64,bRequest= 160, wValue= lV[k], wIndex= 0 , data_or_wLength=  lP[k])